# Portfolio Management Strategy Backtesting

## Introduction
This notebook presents a comprehensive backtest of a dynamic portfolio management strategy designed to achieve bi-directional profitability. The strategy integrates a blend of various asset classes, including index funds, REITs, bonds, leveraged ETFs, options, and cryptocurrencies.
## Methodology
- **Data Sources**: Historical data for each asset class will be sourced from reliable financial databases.
- **Backtesting Framework**: We will use Python libraries such as `pandas`, `numpy`, and `backtrader` for simulation.
- **Performance Metrics**: The strategy's success will be evaluated based on total returns, risk-adjusted returns (Sharpe ratio), drawdown, and income consistency.


## Strategy Overview

### Long-Term Tactics
- **Objective**: Focus on passive income and capital appreciation.
- **Approach**: 
  - Use algorithmic Dollar-Cost Averaging (DCA) to invest in a diversified mix of index funds, REITs, and bonds.
  - Selection of assets is based on favorable macroeconomic indicators and technical analysis.
  - Prioritize assets that demonstrate stability and growth potential over the long term.

### Short-Term Tactics
- **Objective**: Hedge against short-term market volatility and generate regular income.
- **Approach**: 
  - Utilize margin trading to invest in leveraged ETFs, balancing the risk and potential for high returns.
  - Employ credit spread options strategies to generate consistent income.
  - Short-term positions are managed actively, responding to market volatility and opportunities.

In [4]:
!pip install yfinance tradingview-ta mplfinance backtrader requests Backtest Backtesting

In [5]:
# Core Libraries
import numpy as np
import pandas as pd
import yfinance as yf
# Testing Software
import backtrader as bt
# Plotting Software
import plotly.graph_objs as go

## Implementation

### Data Preparation
- Import necessary libraries and load historical data.
- Data preprocessing including normalization, handling missing values, etc.
-

### Short-Term Tactics
- Short term tacts should must account for wash trading rule.
- Options should generate income

In [6]:
hourly_data = yf.download('SPY', start="2023-01-25", end="2023-02-07", interval = "1h")
hourly_data.drop(["Adj Close", "Volume"], inplace=True, axis=1)
hourly_data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close
Datetime,,,,
2023-01-25 09:30:00-05:00,395.679993,396.649994,394.220001,394.850006
2023-01-25 10:30:00-05:00,394.850006,396.429993,393.559998,396.088013
2023-01-25 11:30:00-05:00,396.109985,397.725006,395.954987,396.679413
2023-01-25 12:30:00-05:00,396.684998,398.890015,396.660004,398.755005
2023-01-25 13:30:00-05:00,398.760010,400.209991,398.415009,399.320007
...,...,...,...,...
2023-02-06 11:30:00-05:00,411.019989,411.290009,409.329987,409.679993
2023-02-06 12:30:00-05:00,409.650909,410.480011,409.179993,409.730011
2023-02-06 13:30:00-05:00,409.709991,411.049988,409.394989,410.425995


### Long-Term Tactics
- Long term Tactics should avoid long term tax implications 
- Utilize a "Technical Daily Cost Averaging", using a combination of fundemental anyalsis, technical analysis and quantitative analysis techniques using machine learning to optimize the timing of entering positions depending on the specifc asset and features fine-tuned to that asset.

#### Weekly Data
- Open, High, Low, Close, PP, S1, S2, R1, R2, macD,

In [8]:
weekly_data = yf.download(tickers='SPY', start="2010-01-25", end="2023-02-07", period='id', interval='1wk') 
weekly_data.drop( ["Adj Close"], inplace=True, axis=1 )

weekly_data["PP"] = (weekly_data["High"] + weekly_data["Low"] + weekly_data["Close"]) / 3
weekly_data["R1"] = weekly_data["PP"] * 2 - weekly_data["Low"]
weekly_data["S1"] = weekly_data["PP"] * 2 - weekly_data["High"]
weekly_data["R2"] = weekly_data["PP"] + (weekly_data["High"] - weekly_data["Low"])
weekly_data["S2"] = weekly_data["PP"] - (weekly_data["High"] - weekly_data["Low"])

# removes forward bias
weekly_data["PP"] = weekly_data["PP"].shift(1)
weekly_data["R1"] = weekly_data["R1"].shift(1)
weekly_data["R2"] = weekly_data["R2"].shift(1)
weekly_data["S1"] = weekly_data["S1"].shift(1)
weekly_data["S2"] = weekly_data["S2"].shift(1)

# Removes bias
weekly_data.dropna(inplace=True)
# weekly_data = weekly_data[["Open", "High", "Low", "Close","Volume", "PP", "S1", "S2", "R1", "R2"]]
weekly_data = weekly_data[["PP", "S1", "S2", "R1", "R2"]]

# weekly_data
plot_data = pd.merge_asof(hourly_data, weekly_data, left_index=True, right_index=True)
plot_data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,PP,S1,S2,R1,R2
Datetime,,,,,,,,,
2023-01-25 09:30:00-05:00,395.679993,396.649994,394.220001,394.850006,394.456675,388.683339,381.486674,401.653341,407.426676
2023-01-25 10:30:00-05:00,394.850006,396.429993,393.559998,396.088013,394.456675,388.683339,381.486674,401.653341,407.426676
2023-01-25 11:30:00-05:00,396.109985,397.725006,395.954987,396.679413,394.456675,388.683339,381.486674,401.653341,407.426676
2023-01-25 12:30:00-05:00,396.684998,398.890015,396.660004,398.755005,394.456675,388.683339,381.486674,401.653341,407.426676
2023-01-25 13:30:00-05:00,398.760010,400.209991,398.415009,399.320007,394.456675,388.683339,381.486674,401.653341,407.426676
...,...,...,...,...,...,...,...,...,...
2023-02-06 11:30:00-05:00,411.019989,411.290009,409.329987,409.679993,410.313334,402.316671,392.283335,420.346670,428.343333
2023-02-06 12:30:00-05:00,409.650909,410.480011,409.179993,409.730011,410.313334,402.316671,392.283335,420.346670,428.343333
2023-02-06 13:30:00-05:00,409.709991,411.049988,409.394989,410.425995,410.313334,402.316671,392.283335,420.346670,428.343333


### Strategy Simulation
- Define long-term and short-term investment strategies.
- Set up the backtesting environment.
- Simulate the strategies over the historical data period.


In [9]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG



C:\ProgramData\anaconda3\Lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

C:\ProgramData\anaconda3\Lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
C:\ProgramData\anaconda3\Lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),


### Performance Analysis
- Analyze the returns and risk metrics.
- Compare the performance against benchmark indices.
- Assess the effectiveness of the hedging and income generation tactics.



## Conclusion
- Summary of the backtesting results.
- Insights on the strategy's strengths and areas for improvement.
- Future considerations and potential strategy adjustments.
